# 1. Datascraping de rembrandt schilderijen

Hiervoor moeten we eerst de nodige imports doen 

In [2]:
import os
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse

De url waar we de schilderijen van halen steken we in een variabele, ook de folder waar we ze willen opslaan

In [3]:
BASE_URL = "http://www.rembrandtpainting.net/"
# Gebruik het correcte absolute pad
SAVE_FOLDER = r"C:\Projects\DeepLearning_Schilderijnen\datasets\raw\Rembrandt"
print(f"SAVE_FOLDER ingesteld op: {SAVE_FOLDER}")

SAVE_FOLDER ingesteld op: C:\Projects\DeepLearning_Schilderijnen\datasets\raw\Rembrandt


Controleer dat de map wel degelijk bestaat

In [3]:
if not os.path.exists(SAVE_FOLDER):
    os.makedirs(SAVE_FOLDER)
    print(f"Map aangemaakt: {SAVE_FOLDER}")

Nu moeten we een functie schrijven die HTML vanuit de url kan halen en we gebruiken hiervoor Beautiful soup dat een package is die vaak gebruikt wordt voor webscrapers

In [4]:
def get_soup(url):
    """Haalt de HTML op en maakt er een BeautifulSoup object van."""
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        return BeautifulSoup(response.text, 'html.parser')
    except Exception as e:
        print(f"Fout bij ophalen {url}: {e}")
        return None


Hierna schrijven een functie die de schilderijen uit deze html kan halen en ze downloaden

In [5]:
def download_image(img_url, filename):
    """Downloadt een enkele afbeelding."""
    try:
        img_data = requests.get(img_url, timeout=15).content
        path = os.path.join(SAVE_FOLDER, filename)
        
        # Alleen opslaan als het bestand nog niet bestaat of leeg is
        if not os.path.exists(path) or os.path.getsize(path) == 0:
            with open(path, 'wb') as f:
                f.write(img_data)
            print(f"Gedownload: {filename}")
        else:
            print(f"Bestaat al: {filename}")
            
    except Exception as e:
        print(f"Fout bij downloaden {filename}: {e}")


Dit is de functie die alles zal uitvoeren

In [6]:
def scrape_rembrandt():
    print("Start scraping van rembrandtpainting.net...")
    
    # 1. Haal de homepagina op om categorieën te vinden
    soup = get_soup(BASE_URL)
    if not soup:
        return

    # Trefwoorden om categorie-links te vinden
    category_keywords = ["History Painting", "Landscape", "Self Portrait", "Portraits", "Genre", "Etchings"]
    category_links = set()

    # Zoek alle links op de voorpagina
    for a_tag in soup.find_all('a', href=True):
        link_text = a_tag.get_text().strip()
        href = a_tag['href']
        
        if any(keyword.lower() in link_text.lower() for keyword in category_keywords):
            full_url = urljoin(BASE_URL, href)
            category_links.add(full_url)

    print(f"Gevonden categorie-pagina's: {len(category_links)}")

    # 2. Ga elke categorie pagina af
    total_downloaded = 0
    painting_links = set()
    
    for cat_url in category_links:
        print(f"\nBezig met categorie: {cat_url}")
        cat_soup = get_soup(cat_url)
        if not cat_soup:
            continue
            
        # Zoek naar links naar individuele schilderij pagina's
        # Deze eindigen meestal op .htm en zijn geen navigatie links
        for a in cat_soup.find_all('a', href=True):
            href = a.get('href')
            
            # Filter navigatie en externe links
            if not href or href.startswith('http') or href.startswith('#'):
                continue
            
            # We willen .htm links die niet naar categorie overzichtspagina's gaan
            if '.htm' in href and 'a_rembrandt' not in href.lower():
                painting_url = urljoin(cat_url, href)
                painting_links.add(painting_url)
    
    print(f"\nGevonden schilderij pagina's: {len(painting_links)}")
    
    # 3. Bezoek elke individuele schilderij pagina
    for i, painting_url in enumerate(painting_links, 1):
        print(f"\n[{i}/{len(painting_links)}] Bezig met: {painting_url}")
        
        painting_soup = get_soup(painting_url)
        if not painting_soup:
            continue
        
        # Zoek de grootste afbeelding op deze pagina
        images = painting_soup.find_all('img')
        
        for img in images:
            src = img.get('src')
            if not src:
                continue
                
            # Filter rommel uit
            if any(x in src.lower() for x in ['bullet', 'button', 'logo', 'icon', 'arrow', 'spacer', 
                                                'banner', 'ad_', 'artcomposter', 'dvd']):
                continue
            
            # Bouw volledige URL
            image_url = urljoin(painting_url, src)
            
            # Skip als het geen echte afbeelding is
            if not image_url.lower().endswith(('.jpg', '.jpeg', '.png', '.gif')):
                continue
            
            # Genereer bestandsnaam
            filename = os.path.basename(urlparse(image_url).path)
            
            # Voeg prefix toe
            if not filename.lower().startswith('rembrandt'):
                filename = f"rembrandt_{filename}"
            
            # Download
            download_image(image_url, filename)
            total_downloaded += 1
            break  # Neem alleen de eerste (grootste) afbeelding per pagina

    print(f"\n✓ Klaar! Totaal gedownload: {total_downloaded} afbeeldingen")
    print(f"Check de map: {os.path.abspath(SAVE_FOLDER)}")

if __name__ == "__main__":
    scrape_rembrandt()

Start scraping van rembrandtpainting.net...
Gevonden categorie-pagina's: 11

Bezig met categorie: http://www.rembrandtpainting.net/rmbrdnt_selected_etchings/rembrandt_etchings.htm

Bezig met categorie: http://www.rembrandtpainting.net/complete_catalogue/start_landscapes.htm

Bezig met categorie: http://www.rembrandtpainting.net/rembrandt_self_portraits.htm#Self Portraits

Bezig met categorie: http://www.rembrandtpainting.net/complete_catalogue/start_history_two.htm

Bezig met categorie: http://www.rembrandtpainting.net/rmbrndt_etchings/a_rembrandt_etchings.htm

Bezig met categorie: http://www.rembrandtpainting.net/complete_catalogue/start_history.htm

Bezig met categorie: http://www.rembrandtpainting.net/complete_catalogue/start_self_portraits.htm

Bezig met categorie: http://www.rembrandtpainting.net/rembrandt_self_portraits.htm#about

Bezig met categorie: http://www.rembrandtpainting.net/rembrandt_self_portraits.htm

Bezig met categorie: http://www.rembrandtpainting.net/about_rembran

Wat doet dit script?
1. Het scant de hoofdpagina van rembrandtpainting.net.
2. Het zoekt naar links met woorden als "History", "Landscape", "Portrait" (de categorieën van de schilderijen).
3. Het opent elke categorie-pagina en zoekt naar afbeeldingen.
4. Het is slim genoeg om te checken of een plaatje een link is naar een grotere versie (high-res), wat vaak zo is op oude sites.
5. Het filtert "rommel" zoals navigatiepijltjes en logo's weg.
6. Het slaat alles op in datasets/raw/Rembrandt.